# 1.Requirements

In [ ]:
#importing all the necessary modules

import random
import numpy as np
import pandas as pd
from math import sqrt

from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler

import torch
import torch.nn as nn
import torch.utils.data
import torch.optim as optim

# 2.Setting the arguments

In [ ]:
theta = 7
num_epochs = 50
dropout_ratio = 0.5

pattern = 'mnar'
method = 'uniform'

test_size = 0.3
use_cuda = True
batch_size  = 1

# 3.Data Preprocessing

In [ ]:
#mounting drive folder to access the transcriptomics and proteomics dataset

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#mounting drive folder to access the transcriptomics and proteomics dataset

transcriptome = pd.read_csv("/content/drive/MyDrive/DesignP_omics/Transcriptomics.csv")
transcriptome

,Unnamed: 0,Visit,Month,Site,Diagnosis,Sex,Age,APOE,MMSE,Gexp_batch,...,ILMN_3311145,ILMN_3311150,ILMN_3311155,ILMN_3311160,ILMN_3311165,ILMN_3311170,ILMN_3311175,ILMN_3311180,ILMN_3311185,ILMN_3311190
0,DCR00001_1,1.0,0.0,DCR,AD,Female,80.0,E3E4,21.0,2.0,...,74.9,79.5,76.3,74.9,76.3,79.1,72.9,77.6,71.5,80.7
1,DCR00004_1,1.0,0.0,DCR,MCI,Male,78.0,E4E4,27.0,2.0,...,90.0,95.6,92.1,91.9,107.9,86.9,89.0,99.0,86.6,120.8
2,DCR00009_1,1.0,0.0,DCR,CTL,Female,69.0,E3E3,30.0,2.0,...,81.4,83.7,93.4,84.4,97.0,82.6,81.5,85.2,85.2,106.8
3,DCR00013_1,1.0,0.0,DCR,CTL,Male,82.0,E3E3,30.0,2.0,...,82.9,82.1,83.8,81.6,88.9,77.3,82.2,80.7,82.3,94.3
4,DCR00014_1,1.0,0.0,DCR,CTL,Female,75.0,E2E3,30.0,2.0,...,80.8,81.7,79.0,81.0,89.1,81.0,79.9,81.5,81.0,95.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
700,TLSMCI608_1,1.0,0.0,Toulouse,MCI,Male,73.0,E3E3,28.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
701,TLSMCI609_1,1.0,0.0,Toulouse,MCI,Female,63.0,E3E3,28.0,2.0,...,78.4,83.3,86.8,80.8,84.6,83.5,78.9,87.5,83.7,94.3
702,TLSMCI610_1,1.0,0.0,Toulouse,MCI,Male,73.0,E3E3,29.0,NaN,...,75.2,75.1,76.7,75.8,84.4,71.7,79.2,81.4,75.9,87.8
703,TLSMCI611_1,1.0,0.0,Toulouse,MCI,Female,78.0,E3E3,30.0,2.0,...,96.5,101.5,93.2,89.5,97.5,92.4,92.4,98.5,89.6,110.5


In [ ]:
proteome = pd.read_csv("/content/drive/MyDrive/DesignP_omics/Proteomics.csv")
proteome

,Unnamed: 0,Visit,Month,Site,Diagnosis,Sex,Age,APOE,MMSE,P0C0L5,...,P23396,Q86VB7,Q12884,Q8IXJ6,Q13813,Q08945,P09493,P07478,P04818,P98066
0,DCR00001_1,1.0,0.0,DCR,AD,Female,80.0,E3E4,21.0,3716.1,...,842.5,2916.3,4790.8,1204.7,596.2,948.7,965.3,26393.3,714.4,1289.0
1,DCR00007_1,1.0,0.0,DCR,AD,Male,80.0,E3E4,24.0,5676.7,...,534.2,3156.5,5171.8,1796.0,555.5,908.1,933.6,45037.9,762.2,1370.5
2,DCR00023_1,1.0,0.0,DCR,AD,Female,88.0,E3E3,NaN,6935.7,...,530.4,2948.8,3801.5,1161.2,557.2,1070.1,984.7,38593.2,825.6,1328.2
3,DCR00024_1,1.0,0.0,DCR,AD,Female,80.0,E3E4,NaN,4940.4,...,652.7,2600.1,2882.3,1840.7,520.3,901.3,940.7,40413.4,6286.7,992.4
4,DCR00025_1,1.0,0.0,DCR,AD,Male,75.0,E4E4,NaN,9293.7,...,410.4,2045.8,3141.5,832.8,494.1,896.6,900.9,22241.0,729.2,997.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
893,THSMCI062_2,2.0,12.0,Thessaloniki,AD,Male,73.0,NaN,20.0,11673.0,...,1031.6,2380.4,2213.6,1397.2,457.3,920.7,982.7,17674.7,560.3,1137.6
894,THSMCI064_1,1.0,0.0,Thessaloniki,MCI,Male,80.0,E3E4,28.0,7606.8,...,1333.7,1659.9,2264.9,1283.5,483.9,1197.9,863.0,23074.0,561.0,1142.0
895,THSMCI064_2,2.0,12.0,Thessaloniki,AD,Male,80.0,E3E4,22.0,4941.0,...,1078.8,1832.6,2959.2,1597.6,541.8,1281.8,843.8,23816.7,565.0,1161.1
896,THSMCI065_1,1.0,0.0,Thessaloniki,MCI,Male,76.0,E3E3,29.0,13515.5,...,1816.2,2861.4,2809.6,1897.8,567.2,1013.0,943.1,37664.4,629.0,1209.7


In [ ]:
proteome = proteome.drop(["Visit","Month","Site","Diagnosis","Sex","Age","APOE","MMSE"],axis=1)
proteome

,Unnamed: 0,P0C0L5,P03951,Q9Y4X3,P39060,P01033,P00750,P58294,P39905,P16035,...,P23396,Q86VB7,Q12884,Q8IXJ6,Q13813,Q08945,P09493,P07478,P04818,P98066
0,DCR00001_1,3716.1,2705.5,908.1,5013.4,5368.9,1702.5,515.9,1302.5,6102.3,...,842.5,2916.3,4790.8,1204.7,596.2,948.7,965.3,26393.3,714.4,1289.0
1,DCR00007_1,5676.7,3217.3,869.1,6088.9,5487.7,1052.4,492.4,1272.7,7670.6,...,534.2,3156.5,5171.8,1796.0,555.5,908.1,933.6,45037.9,762.2,1370.5
2,DCR00023_1,6935.7,2457.9,848.7,7136.7,6462.7,1256.2,517.8,1197.9,6104.7,...,530.4,2948.8,3801.5,1161.2,557.2,1070.1,984.7,38593.2,825.6,1328.2
3,DCR00024_1,4940.4,2779.3,888.9,6622.9,4846.0,1930.0,512.8,1210.5,5383.9,...,652.7,2600.1,2882.3,1840.7,520.3,901.3,940.7,40413.4,6286.7,992.4
4,DCR00025_1,9293.7,2199.7,782.2,6391.1,5344.6,1405.3,509.0,1254.8,5644.1,...,410.4,2045.8,3141.5,832.8,494.1,896.6,900.9,22241.0,729.2,997.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
893,THSMCI062_2,11673.0,2234.2,843.2,4923.5,5025.3,1101.1,490.8,1132.6,5187.2,...,1031.6,2380.4,2213.6,1397.2,457.3,920.7,982.7,17674.7,560.3,1137.6
894,THSMCI064_1,7606.8,2815.7,893.8,5035.3,5625.3,1216.0,485.9,1160.6,5246.9,...,1333.7,1659.9,2264.9,1283.5,483.9,1197.9,863.0,23074.0,561.0,1142.0
895,THSMCI064_2,4941.0,2734.2,892.1,5240.2,5942.1,1089.3,472.8,1186.5,6573.6,...,1078.8,1832.6,2959.2,1597.6,541.8,1281.8,843.8,23816.7,565.0,1161.1
896,THSMCI065_1,13515.5,2013.5,865.0,7424.7,6206.0,1482.8,539.3,1306.6,4602.7,...,1816.2,2861.4,2809.6,1897.8,567.2,1013.0,943.1,37664.4,629.0,1209.7


In [ ]:
count = 0
for i in transcriptome["Unnamed: 0"]:
    for j in proteome["Unnamed: 0"]:
        if(i==j):
            count+=1
print(count)

345


In [ ]:
transcriptome = transcriptome.fillna(0)

In [ ]:
df = pd.merge(proteome, transcriptome, on="Unnamed: 0")
df

,Unnamed: 0,P0C0L5,P03951,Q9Y4X3,P39060,P01033,P00750,P58294,P39905,P16035,...,ILMN_3311145,ILMN_3311150,ILMN_3311155,ILMN_3311160,ILMN_3311165,ILMN_3311170,ILMN_3311175,ILMN_3311180,ILMN_3311185,ILMN_3311190
0,DCR00001_1,3716.1,2705.5,908.1,5013.4,5368.9,1702.5,515.9,1302.5,6102.3,...,74.9,79.5,76.3,74.9,76.3,79.1,72.9,77.6,71.5,80.7
1,DCR00023_1,6935.7,2457.9,848.7,7136.7,6462.7,1256.2,517.8,1197.9,6104.7,...,81.2,81.4,84.7,79.4,83.8,78.3,78.6,81.3,81.6,91.4
2,DCR00025_1,9293.7,2199.7,782.2,6391.1,5344.6,1405.3,509.0,1254.8,5644.1,...,84.9,82.2,76.5,84.5,88.0,77.1,85.4,87.3,86.1,96.8
3,DCR00028_1,9874.5,2327.7,857.7,6675.4,4911.2,1473.6,491.8,1380.8,6057.6,...,81.1,85.7,80.5,79.7,87.0,77.7,79.8,79.7,78.9,97.9
4,DCR00031_1,15269.8,2215.4,869.1,7354.2,5412.8,1188.3,569.8,1159.9,6387.7,...,92.9,99.0,101.3,96.6,97.6,97.6,98.2,95.1,98.7,114.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
340,THSMCI060_1,6871.7,1971.9,877.6,3747.0,4831.4,1108.8,519.4,1090.7,4606.3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
341,THSMCI061_1,7369.0,2663.2,866.3,3981.2,4977.6,1302.5,468.7,1346.4,5587.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
342,THSMCI062_1,10184.5,1996.0,869.3,4510.9,5501.0,1197.8,497.3,1229.8,4395.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
343,THSMCI064_1,7606.8,2815.7,893.8,5035.3,5625.3,1216.0,485.9,1160.6,5246.9,...,70.8,71.1,75.6,75.4,74.7,70.8,70.6,74.7,72.1,77.6


In [ ]:
data = df.drop(["Unnamed: 0","Visit","Month","Site","Diagnosis","Sex","Age","APOE","MMSE","Gexp_batch"],axis=1)
data

,P0C0L5,P03951,Q9Y4X3,P39060,P01033,P00750,P58294,P39905,P16035,P48061,...,ILMN_3311145,ILMN_3311150,ILMN_3311155,ILMN_3311160,ILMN_3311165,ILMN_3311170,ILMN_3311175,ILMN_3311180,ILMN_3311185,ILMN_3311190
0,3716.1,2705.5,908.1,5013.4,5368.9,1702.5,515.9,1302.5,6102.3,4140.8,...,74.9,79.5,76.3,74.9,76.3,79.1,72.9,77.6,71.5,80.7
1,6935.7,2457.9,848.7,7136.7,6462.7,1256.2,517.8,1197.9,6104.7,4385.7,...,81.2,81.4,84.7,79.4,83.8,78.3,78.6,81.3,81.6,91.4
2,9293.7,2199.7,782.2,6391.1,5344.6,1405.3,509.0,1254.8,5644.1,3933.9,...,84.9,82.2,76.5,84.5,88.0,77.1,85.4,87.3,86.1,96.8
3,9874.5,2327.7,857.7,6675.4,4911.2,1473.6,491.8,1380.8,6057.6,3991.9,...,81.1,85.7,80.5,79.7,87.0,77.7,79.8,79.7,78.9,97.9
4,15269.8,2215.4,869.1,7354.2,5412.8,1188.3,569.8,1159.9,6387.7,3960.3,...,92.9,99.0,101.3,96.6,97.6,97.6,98.2,95.1,98.7,114.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
340,6871.7,1971.9,877.6,3747.0,4831.4,1108.8,519.4,1090.7,4606.3,4337.7,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
341,7369.0,2663.2,866.3,3981.2,4977.6,1302.5,468.7,1346.4,5587.5,3754.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
342,10184.5,1996.0,869.3,4510.9,5501.0,1197.8,497.3,1229.8,4395.5,3975.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
343,7606.8,2815.7,893.8,5035.3,5625.3,1216.0,485.9,1160.6,5246.9,3630.8,...,70.8,71.1,75.6,75.4,74.7,70.8,70.6,74.7,72.1,77.6


In [ ]:
data = data.head(100)
data = data[data.columns[:10000]]
data

,P0C0L5,P03951,Q9Y4X3,P39060,P01033,P00750,P58294,P39905,P16035,P48061,...,ILMN_1694078,ILMN_1694080,ILMN_1694082,ILMN_1694084,ILMN_1694100,ILMN_1694106,ILMN_1694111,ILMN_1694126,ILMN_1694127,ILMN_1694131
0,3716.1,2705.5,908.1,5013.4,5368.9,1702.5,515.9,1302.5,6102.3,4140.8,...,77.30000,75.10000,75.50000,574.700000,115.2000,105.4000,144.600000,69.9000,80.0000,77.50000
1,6935.7,2457.9,848.7,7136.7,6462.7,1256.2,517.8,1197.9,6104.7,4385.7,...,80.90000,76.50000,80.60000,636.800000,78.6000,193.1000,298.000000,80.4000,82.6000,76.90000
2,9293.7,2199.7,782.2,6391.1,5344.6,1405.3,509.0,1254.8,5644.1,3933.9,...,80.50000,79.90000,82.60000,831.600000,85.6000,121.9000,191.500000,79.5000,95.1000,79.90000
3,9874.5,2327.7,857.7,6675.4,4911.2,1473.6,491.8,1380.8,6057.6,3991.9,...,77.40000,74.60000,74.50000,1645.800000,99.4000,270.7000,195.200000,79.4000,98.3000,77.20000
4,15269.8,2215.4,869.1,7354.2,5412.8,1188.3,569.8,1159.9,6387.7,3960.3,...,101.00000,99.30000,89.40000,1518.200000,103.9000,209.3000,281.600000,91.4000,119.9000,92.60000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,5404.5,2408.0,823.5,6067.5,5219.2,1365.1,499.1,1383.4,5810.8,3733.4,...,147.35780,130.78910,154.64670,6844.425994,146.2730,577.8992,1093.637998,140.8410,217.5629,147.39580
96,3625.8,2023.0,783.3,4406.6,3777.0,1062.7,484.1,1420.9,7619.3,3977.8,...,143.32670,139.85220,131.06600,9347.564010,141.8790,992.6697,1198.542000,164.2425,294.9116,121.87080
97,11337.7,2128.3,843.9,6162.4,5071.5,953.0,480.4,1151.5,6408.4,3736.4,...,97.18447,90.65845,92.35209,3379.115996,102.0947,303.2480,500.281500,102.7871,135.6408,93.79276
98,5102.9,2605.9,914.6,7109.9,5126.6,840.1,513.6,1121.5,6155.0,3196.5,...,83.70000,80.40000,80.10000,1025.800000,98.0000,182.9000,191.600000,81.4000,84.0000,79.90000


In [ ]:
#data preprocessing
rows, cols = data.shape
shuffled_index = np.random.permutation(rows)
train_index = shuffled_index[:int(rows*(1-test_size))]
test_index = shuffled_index[int(rows*(1-test_size)):]

train_data = data.iloc[train_index, :]
test_data = data.iloc[test_index, :]

print(test_data)

# standardized between 0 and 1
scaler = MinMaxScaler()
scaler.fit(train_data)
train_data = scaler.transform(train_data)
test_data = scaler.transform(test_data)

     P0C0L5  P03951  Q9Y4X3  P39060  P01033  P00750  P58294  P39905  P16035  \
44  17707.5  2169.0   751.3  7591.5  8901.9  1565.4   447.3  1348.8  6607.0   
63  12680.3  2015.9   776.4  5398.1  4907.8  1284.8   506.8  1434.8  5343.9   
18   7051.4  2050.5   864.8  5157.5  4774.1  1166.5   508.8  1208.4  5848.3   
14   8797.1  2694.4   916.0  4772.3  5308.3  1563.9   531.1  1200.7  4685.3   
64   8911.4  2071.9  1279.0  4785.1  4825.7   929.0   485.5  1050.9  6525.0   
23  12508.7  2069.9   849.8  4440.2  4448.7  1568.5   498.9  1302.7  4749.7   
16  11436.6  2708.6   825.0  5576.4  4730.7  1260.4   483.7  1209.6  6616.8   
80   7814.5  2437.4   966.7  7565.6  4185.0  1389.7   533.8  1091.1  6039.3   
92   2662.9  2187.5   898.8  8201.4  6021.0  1228.1   492.8  1220.4  6395.9   
37   6137.2  1820.9   960.2  5508.8  4789.0   797.9   528.1  1033.2  9227.9   
17   4139.9  2348.3   918.4  5254.2  3895.9   942.2   503.9  1157.3  6640.2   
83  11283.4  2658.7   889.0  5896.1  4381.3  1118.2 

In [ ]:
print(rows)
print(cols)

100
10000


In [ ]:
# function to generate missingness in the data; MCAR = Missing completely at random, MNAR = Missing not at random
def generate_missingness(test_data, pattern, method):
    rows, cols = test_data.shape
    newdata = test_data.copy()
    mask=np.empty([rows,cols],dtype=bool)
    # missingness threshold
    t = 0.2

    if(pattern == 'mcar'):

        if(method == 'uniform'):
            # uniform random vector
            v = np.random.uniform(size=(rows, cols))
            r,c = v.shape
            for i in range(r):
                for j in range(c):
                    if(v[i][j]<=t):    #assign all cells to -1 where the value of v is less than 0.2
                        newdata[i][j] = 0
                        mask[i][j] = True
                    else:
                        mask[i][j] = False

        elif(method == 'random'):
            missing_cols = np.random.choice(cols, cols//2)    #selecting half of the columns
            temp = np.zeros(cols, dtype=bool)
            temp[missing_cols] = True
            v = np.random.uniform(size=(rows, cols))
            r,c = v.shape
            for i in range(r):
                for j in range(c):
                    if(v[i][j]<=t and temp[j]):    #assign all cells to -1 where the value of v is less than 0.2
                        newdata[i][j] = 0      #and the column is one of the columns chosen above for missingness
                        mask[i][j] = True
                    else:
                        mask[i][j] = False
        else:
            print('Incorrect Input')

    elif(pattern == 'mnar'):

        if(method == 'uniform'):
            sample_cols = np.random.choice(cols, 2)    #choosing any 2 attributes
            m1,m2 = np.median(newdata[:,sample_cols], axis=0)    #calculating the median of the 2 chosen columns
            v = np.random.uniform(size=(rows, cols))
            # to delete values where (v<=t) and (x1 <= m1 or x2 >= m2)
            m1 = newdata[:,sample_cols[0]] <= m1
            m2 = newdata[:,sample_cols[1]] >= m2
            m = (m1*m2)[:, np.newaxis]
            mask = m* (v<=t)    #assign all cells to -1 where the value of v is less than 0.2
            newdata[mask]=0           #and x1 <= m1 or x2 >= m2

        elif(method == 'random'):
            missing_cols = np.random.choice(cols, cols//2)    #selecting half of the columns
            temp = np.zeros(cols, dtype=bool)
            temp[missing_cols] = True
            sample_cols = np.random.choice(cols, 2)    #choosing any 2 attributes
            m1,m2 = np.median(newdata[:,sample_cols], axis=0)    #calculating the median of the 2 chosen columns
            v = np.random.uniform(size=(rows, cols))
            # to select rows where (x1 <= m1 or x2 >= m2)
            m1 = newdata[:,sample_cols[0]] <= m1
            m2 = newdata[:,sample_cols[1]] >= m2
            m = (m1*m2)[:, np.newaxis]
            mask = m* (v<=t) * temp    #assign all cells to -1 where the value of v is less than 0.2
            newdata[mask]=0           #and x1 <= m1 or x2 >= m2
        else:
            print("Incorrect Input")
    else:
        print("Incorrect Input")
    return newdata, mask

# 4.Model

In [ ]:
missed_data, mask = generate_missingness(test_data, pattern=pattern, method=method)

In [ ]:
#to print the number of missing datapoints
count=0
for i in range(len(missed_data)):
    for j in range(len(missed_data[0])):
        if(missed_data[i][j]==-1):
            count+=1
print(count)

0


In [ ]:
missed_data

array([[ 8.16463698e-01,  4.68996416e-01,  6.79809653e-04, ...,
         4.35887758e-01,  2.13720589e-01,  4.13321116e-01],
       [ 5.65245636e-01,  3.77538829e-01,  8.59959211e-02, ...,
         9.01225005e-01,  4.90548402e-01,  5.52856492e-01],
       [ 2.83959563e-01,  3.98207885e-01,  3.86471788e-01, ...,
         4.64527526e-01,  3.27622003e-01,  4.21405947e-01],
       ...,
       [ 1.17288732e-01,  7.89486260e-01,  5.33650578e-01, ...,
        -3.51098554e-02,  4.65814537e-02,  6.46193286e-02],
       [ 1.53123485e-01,  5.12485066e-01,  3.04554725e-01, ...,
         4.89875744e-01,  2.24972007e-01,  4.41103190e-01],
       [ 6.12234088e-01,  5.32795699e-01,  1.81169273e-01, ...,
         2.68556601e-01,  2.16268373e-01,  2.69478067e-01]])

In [ ]:
mask

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [ ]:
missed_data = torch.from_numpy(missed_data).float()
train_data = torch.from_numpy(train_data).float()

train_loader = torch.utils.data.DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)

In [ ]:
device = torch.device("cuda" if use_cuda else "cpu")

In [ ]:
class Autoencoder(nn.Module):
    def __init__(self, dim):
        super(Autoencoder, self).__init__()
        self.dim = dim
        
        self.drop_out = nn.Dropout(p=0.5)
        
        self.encoder = nn.Sequential(
            nn.Linear(dim+theta*0, dim+theta*1),
            nn.Tanh(),
            nn.Linear(dim+theta*1, dim+theta*2),
            nn.Tanh(),
            nn.Linear(dim+theta*2, dim+theta*3)
        )
            
        self.decoder = nn.Sequential(
            nn.Linear(dim+theta*3, dim+theta*2),
            nn.Tanh(),
            nn.Linear(dim+theta*2, dim+theta*1),
            nn.Tanh(),
            nn.Linear(dim+theta*1, dim+theta*0)
        )
        
    def forward(self, x):
        x = x.view(-1, self.dim)
        x_missed = self.drop_out(x)
        
        z = self.encoder(x_missed)
        out = self.decoder(z)
        
        out = out.view(-1, self.dim)
        
        return out

In [ ]:
model = Autoencoder(dim=cols).to(device)

In [ ]:
loss = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), momentum=0.99, lr=0.01, nesterov=True)

# 5.Training phase

In [ ]:
cost_list = []
early_stop = False

for epoch in range(num_epochs):
    
    total_batch = len(train_data) // batch_size
    
    for i, batch_data in enumerate(train_loader):
        
        batch_data = batch_data.to(device)
        
        reconst_data = model(batch_data)
        cost = loss(reconst_data, batch_data)
        
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()
                
        if (i+1) % (total_batch//2) == 0:
            print('Epoch [%d/%d], lter [%d/%d], Loss: %.6f'
                 %(epoch+1, num_epochs, i+1, total_batch, cost.item()))
            
        # early stopping rule 1 : MSE < 1e-06
        if cost.item() < 1e-06 :
            early_stop = True
            break
                            
        cost_list.append(cost.item())

    if early_stop :
        break
        
print("Learning Finished!")

Epoch [1/50], lter [35/70], Loss: 0.270361
Epoch [1/50], lter [70/70], Loss: 0.060982
Epoch [2/50], lter [35/70], Loss: 0.062833
Epoch [2/50], lter [70/70], Loss: 0.043188
Epoch [3/50], lter [35/70], Loss: 0.030208
Epoch [3/50], lter [70/70], Loss: 0.038339
Epoch [4/50], lter [35/70], Loss: 0.020830
Epoch [4/50], lter [70/70], Loss: 0.013418
Epoch [5/50], lter [35/70], Loss: 0.011919
Epoch [5/50], lter [70/70], Loss: 0.012744
Epoch [6/50], lter [35/70], Loss: 0.009417
Epoch [6/50], lter [70/70], Loss: 0.021554
Epoch [7/50], lter [35/70], Loss: 0.020832
Epoch [7/50], lter [70/70], Loss: 0.023423
Epoch [8/50], lter [35/70], Loss: 0.016165
Epoch [8/50], lter [70/70], Loss: 0.014230
Epoch [9/50], lter [35/70], Loss: 0.010461
Epoch [9/50], lter [70/70], Loss: 0.009911
Epoch [10/50], lter [35/70], Loss: 0.024030
Epoch [10/50], lter [70/70], Loss: 0.036995
Epoch [11/50], lter [35/70], Loss: 0.026223
Epoch [11/50], lter [70/70], Loss: 0.019259
Epoch [12/50], lter [35/70], Loss: 0.005483
Epoch 

# 6.Testing phase

In [ ]:
model.eval()
filled_data = model(missed_data.to(device))
filled_data = filled_data.cpu().detach().numpy()

rmse_sum = 0
rmse_count = 0
for i in range(cols) :
    if mask[:,i].sum() > 0 :
        y_actual = test_data[:,i][mask[:,i]]
        y_predicted = filled_data[:,i][mask[:,i]]

        rmse = sqrt(mean_squared_error(y_actual, y_predicted))
        rmse_sum += rmse
        rmse_count += 1
print("RMSE_SUM :", rmse_sum)
print("Average RMSE :", rmse_sum/rmse_count)

RMSE_SUM : 337.4286533925493
Average RMSE : 0.09224402771802878


In [ ]:
missed_list = missed_data.numpy()

In [ ]:
missed_df = pd.DataFrame(missed_list)
missed_df

,0,1,2,3,4,5,6,7,8,9,...,9990,9991,9992,9993,9994,9995,9996,9997,9998,9999
0,0.816464,0.468996,0.000680,0.433596,0.665472,0.342203,-0.001344,0.407164,0.421115,0.422630,...,0.430827,0.627959,0.387866,0.233494,0.348974,0.209175,0.236694,0.435888,0.213721,0.413321
1,0.565246,0.377539,0.085996,0.218869,0.164914,0.209814,0.112882,0.511940,0.159901,0.498875,...,0.982196,1.142184,0.631033,0.554271,0.684011,0.786335,0.485507,0.901225,0.490548,0.552857
2,0.283960,0.398208,0.386472,0.195315,0.148158,0.153999,0.116721,0.236111,0.264213,0.240036,...,0.483280,0.587259,0.585260,0.344144,0.365154,0.393162,0.247397,0.464528,0.327622,0.421406
3,0.371195,0.782855,0.560503,0.157605,0.215107,0.341496,0.159532,0.226730,0.023700,0.445841,...,0.211661,0.164097,0.053319,0.151350,0.253205,0.222977,0.246241,0.084483,0.128210,0.111230
4,0.000000,0.000000,0.000000,0.158858,0.000000,0.000000,0.071991,0.044225,0.404157,0.417378,...,0.434540,0.719933,0.544063,0.458614,0.269118,0.578060,0.340859,0.425829,0.638942,0.569669
5,0.556670,0.409797,0.335486,0.125093,0.107378,0.343666,0.097715,0.350999,0.037018,0.493248,...,0.756709,0.883109,0.605049,0.526386,0.872603,0.986474,0.542608,0.631705,0.526079,0.639559
6,0.503096,0.791338,0.251190,0.236324,0.142719,0.198301,0.068535,0.237573,0.423141,-0.137063,...,0.804376,0.881721,0.831560,0.693847,0.785904,0.827627,0.566742,0.810071,0.931535,0.522158
7,0.322093,0.629331,0.732835,0.431061,0.074330,0.259306,0.164715,0.093202,0.303712,0.007362,...,0.658044,0.864822,0.483596,0.519626,0.673052,0.477407,0.617795,0.475992,0.263124,0.555573
8,0.064658,0.480048,0.502039,0.493304,0.304425,0.183062,0.086005,0.250731,0.377458,0.304230,...,0.623235,0.932994,0.444422,0.574786,0.560629,0.631869,0.527905,0.622468,0.817982,0.463800
9,0.238275,0.261051,0.710741,0.229706,0.150026,-0.019910,0.153772,0.022661,0.963127,0.611132,...,0.651885,0.983406,0.668124,0.445096,0.513713,0.277209,0.885974,0.592909,0.435276,0.540925


In [ ]:
imputed_list = missed_list.copy()
for i in range(len(missed_list)):
    for j in range(len(missed_list[0])):
        if(missed_list[i][j] == -1):
            imputed_list[i][j] = filled_data[i][j]
        else:
            imputed_list[i][j] = missed_data[i][j]

In [ ]:
filled_df = pd.DataFrame(imputed_list)
filled_df

,0,1,2,3,4,5,6,7,8,9,...,9990,9991,9992,9993,9994,9995,9996,9997,9998,9999
0,0.816464,0.468996,0.000680,0.433596,0.665472,0.342203,-0.001344,0.407164,0.421115,0.422630,...,0.430827,0.627959,0.387866,0.233494,0.348974,0.209175,0.236694,0.435888,0.213721,0.413321
1,0.565246,0.377539,0.085996,0.218869,0.164914,0.209814,0.112882,0.511940,0.159901,0.498875,...,0.982196,1.142184,0.631033,0.554271,0.684011,0.786335,0.485507,0.901225,0.490548,0.552857
2,0.283960,0.398208,0.386472,0.195315,0.148158,0.153999,0.116721,0.236111,0.264213,0.240036,...,0.483280,0.587259,0.585260,0.344144,0.365154,0.393162,0.247397,0.464528,0.327622,0.421406
3,0.371195,0.782855,0.560503,0.157605,0.215107,0.341496,0.159532,0.226730,0.023700,0.445841,...,0.211661,0.164097,0.053319,0.151350,0.253205,0.222977,0.246241,0.084483,0.128210,0.111230
4,0.000000,0.000000,0.000000,0.158858,0.000000,0.000000,0.071991,0.044225,0.404157,0.417378,...,0.434540,0.719933,0.544063,0.458614,0.269118,0.578060,0.340859,0.425829,0.638942,0.569669
5,0.556670,0.409797,0.335486,0.125093,0.107378,0.343666,0.097715,0.350999,0.037018,0.493248,...,0.756709,0.883109,0.605049,0.526386,0.872603,0.986474,0.542608,0.631705,0.526079,0.639559
6,0.503096,0.791338,0.251190,0.236324,0.142719,0.198301,0.068535,0.237573,0.423141,-0.137063,...,0.804376,0.881721,0.831560,0.693847,0.785904,0.827627,0.566742,0.810071,0.931535,0.522158
7,0.322093,0.629331,0.732835,0.431061,0.074330,0.259306,0.164715,0.093202,0.303712,0.007362,...,0.658044,0.864822,0.483596,0.519626,0.673052,0.477407,0.617795,0.475992,0.263124,0.555573
8,0.064658,0.480048,0.502039,0.493304,0.304425,0.183062,0.086005,0.250731,0.377458,0.304230,...,0.623235,0.932994,0.444422,0.574786,0.560629,0.631869,0.527905,0.622468,0.817982,0.463800
9,0.238275,0.261051,0.710741,0.229706,0.150026,-0.019910,0.153772,0.022661,0.963127,0.611132,...,0.651885,0.983406,0.668124,0.445096,0.513713,0.277209,0.885974,0.592909,0.435276,0.540925
